<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas
`Fecha de entrega: 🎃 Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Arturo Hernandez\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
path = 'C:/Python/NLP/nlp_projects/Taller_8/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['Pre-Procesado'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [10]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Procesado'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['Pre-Procesado'].values]

#Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=100)

In [11]:
model.print_topics(num_words=15)

[(0,
  '0.045*"juego" + 0.020*"ps" + 0.019*"bien" + 0.018*"perfectamente" + 0.018*"si" + 0.017*"producto" + 0.016*"cumple" + 0.016*"perfecto" + 0.015*"ser" + 0.014*"buen" + 0.013*"compra" + 0.013*"jugar" + 0.013*"mando" + 0.012*"español" + 0.012*"viene"'),
 (1,
  '0.066*"calidad" + 0.046*"bien" + 0.042*"precio" + 0.035*"buena" + 0.025*"bastante" + 0.022*"buen" + 0.017*"puede" + 0.017*"bonitos" + 0.016*"pendientes" + 0.015*"tamaño" + 0.015*"aunque" + 0.013*"bonito" + 0.012*"plata" + 0.012*"diseño" + 0.011*"queda"'),
 (2,
  '0.034*"bien" + 0.028*"regalo" + 0.026*"caja" + 0.023*"cadena" + 0.020*"día" + 0.019*"colgante" + 0.018*"llegado" + 0.015*"pulsera" + 0.014*"viene" + 0.013*"parece" + 0.011*"ve" + 0.011*"si" + 0.010*"artículo" + 0.010*"bonita" + 0.010*"uso"'),
 (3,
  '0.026*"bonito" + 0.025*"mas" + 0.024*"juego" + 0.020*"calidad" + 0.020*"regalo" + 0.016*"demasiado" + 0.016*"dos" + 0.015*"bien" + 0.015*"si" + 0.014*"sido" + 0.013*"entrega" + 0.012*"días" + 0.012*"llego" + 0.010*"compr

###  `[25 pts]` Punto 3: Visualización de LDA

In [12]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Arturo Hernandez\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
